# 109 persons
## electrodes : 8, 16, 32, 64

In [2]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2023-10-04 15:14:13.263294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 15:14:18.392246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [3]:
no_of_patients=109

In [4]:
train=sorted(glob('files2/*.edf'))
train=train[:no_of_patients]
train

['files2/S001R04.edf',
 'files2/S002R04.edf',
 'files2/S003R04.edf',
 'files2/S004R04.edf',
 'files2/S005R04.edf',
 'files2/S006R04.edf',
 'files2/S007R04.edf',
 'files2/S008R04.edf',
 'files2/S009R04.edf',
 'files2/S010R04.edf',
 'files2/S011R04.edf',
 'files2/S012R04.edf',
 'files2/S013R04.edf',
 'files2/S014R04.edf',
 'files2/S015R04.edf',
 'files2/S016R04.edf',
 'files2/S017R04.edf',
 'files2/S018R04.edf',
 'files2/S019R04.edf',
 'files2/S020R04.edf',
 'files2/S021R04.edf',
 'files2/S022R04.edf',
 'files2/S023R04.edf',
 'files2/S024R04.edf',
 'files2/S025R04.edf',
 'files2/S026R04.edf',
 'files2/S027R04.edf',
 'files2/S028R04.edf',
 'files2/S029R04.edf',
 'files2/S030R04.edf',
 'files2/S031R04.edf',
 'files2/S032R04.edf',
 'files2/S033R04.edf',
 'files2/S034R04.edf',
 'files2/S035R04.edf',
 'files2/S036R04.edf',
 'files2/S037R04.edf',
 'files2/S038R04.edf',
 'files2/S039R04.edf',
 'files2/S040R04.edf',
 'files2/S041R04.edf',
 'files2/S042R04.edf',
 'files2/S043R04.edf',
 'files2/S0

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(1308000, 163391, 1308000, 163391)

In [10]:
print(xtest.iloc[:,-1].values)

[-1.0e-05  1.5e-05  6.0e-06 ...  4.0e-06  1.2e-05 -3.0e-06]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000077,0.000051,0.000020,-0.000052,-0.000064,0.000143,-0.000131,0.000119,-0.000013,-0.000012,...,0.000050,-0.000005,-0.000011,-0.000044,0.000001,0.000042,0.000003,0.000048,-0.000040,-0.000022
1307996,0.000071,0.000063,0.000092,-0.000048,-0.000063,0.000181,-0.000130,0.000136,-0.000006,0.000016,...,0.000052,0.000007,0.000006,-0.000023,0.000016,0.000049,0.000020,0.000069,-0.000026,-0.000004
1307997,0.000062,0.000065,0.000119,-0.000045,-0.000058,0.000166,-0.000122,0.000145,-0.000007,0.000020,...,0.000042,-0.000002,0.000001,-0.000022,0.000010,0.000039,0.000013,0.000067,-0.000029,-0.000012
1307998,0.000052,0.000061,0.000111,-0.000049,-0.000056,0.000152,-0.000108,0.000123,-0.000017,0.000011,...,0.000038,-0.000011,-0.000006,-0.000029,0.000004,0.000036,0.000008,0.000057,-0.000035,-0.000014


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_1457/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_1457/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_1457/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000077,0.000051,0.000020,-0.000052,-0.000064,0.000143,-0.000131,0.000119,-0.000013,-0.000012,...,0.000050,-0.000005,-0.000011,-0.000044,0.000001,0.000042,0.000003,0.000048,-0.000040,-0.000022
1307996,0.000071,0.000063,0.000092,-0.000048,-0.000063,0.000181,-0.000130,0.000136,-0.000006,0.000016,...,0.000052,0.000007,0.000006,-0.000023,0.000016,0.000049,0.000020,0.000069,-0.000026,-0.000004
1307997,0.000062,0.000065,0.000119,-0.000045,-0.000058,0.000166,-0.000122,0.000145,-0.000007,0.000020,...,0.000042,-0.000002,0.000001,-0.000022,0.000010,0.000039,0.000013,0.000067,-0.000029,-0.000012
1307998,0.000052,0.000061,0.000111,-0.000049,-0.000056,0.000152,-0.000108,0.000123,-0.000017,0.000011,...,0.000038,-0.000011,-0.000006,-0.000029,0.000004,0.000036,0.000008,0.000057,-0.000035,-0.000014


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
40875/40875 [==============================] - 58s 1ms/step - loss: 3.0233 - val_loss: 3.2474
Epoch 2/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.5511 - val_loss: 3.2947
Epoch 3/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.4391 - val_loss: 3.2531
Epoch 4/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3907 - val_loss: 3.3634
Epoch 5/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3591 - val_loss: 3.3301
Epoch 6/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3356 - val_loss: 3.3240
Epoch 7/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3193 - val_loss: 3.5077
Epoch 8/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.3059 - val_loss: 3.3019
Epoch 9/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.2955 - val_loss: 3.3437
Epoch 10/10
40875/40875 [==============================

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

  61/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 4s 829us/step
              precision    recall  f1-score   support

           0       0.16      0.12      0.13      1499
           1       0.20      0.22      0.21      1499
           2       0.16      0.22      0.19      1499
           3       0.34      0.51      0.41      1499
           4       0.59      0.68      0.63      1499
           5       0.22      0.23      0.22      1499
           6       0.19      0.46      0.26      1499
           7       0.32      0.28      0.30      1499
           8       0.34      0.15      0.21      1499
           9       0.25      0.41      0.31      1499
          10       0.15      0.35      0.21      1499
          11       0.08      0.10      0.09      1499
          12       0.04      0.00      0.01      1499
          13       0.12      0.11      0.11      1499
          14       0.13      0.03      0.04      1499
          15       0.26      0.15      0.19      1499
          16       0.1

## 0-16

In [19]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [20]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_1457/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_1457/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_1457/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000077,0.000051,0.000020,-0.000052,-0.000064,0.000143,-0.000131,0.000119,-0.000013,-0.000012,...,0.000050,-0.000005,-0.000011,-0.000044,0.000001,0.000042,0.000003,0.000048,-0.000040,-0.000022
1307996,0.000071,0.000063,0.000092,-0.000048,-0.000063,0.000181,-0.000130,0.000136,-0.000006,0.000016,...,0.000052,0.000007,0.000006,-0.000023,0.000016,0.000049,0.000020,0.000069,-0.000026,-0.000004
1307997,0.000062,0.000065,0.000119,-0.000045,-0.000058,0.000166,-0.000122,0.000145,-0.000007,0.000020,...,0.000042,-0.000002,0.000001,-0.000022,0.000010,0.000039,0.000013,0.000067,-0.000029,-0.000012
1307998,0.000052,0.000061,0.000111,-0.000049,-0.000056,0.000152,-0.000108,0.000123,-0.000017,0.000011,...,0.000038,-0.000011,-0.000006,-0.000029,0.000004,0.000036,0.000008,0.000057,-0.000035,-0.000014


In [21]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10


40875/40875 [==============================] - 56s 1ms/step - loss: 2.2923 - val_loss: 2.8555
Epoch 2/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.6298 - val_loss: 2.9743
Epoch 3/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.4980 - val_loss: 2.9914
Epoch 4/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.4142 - val_loss: 2.9620
Epoch 5/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.3614 - val_loss: 2.9711
Epoch 6/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.3288 - val_loss: 2.9857
Epoch 7/10
40875/40875 [==============================] - 56s 1ms/step - loss: 1.3042 - val_loss: 3.1356
Epoch 8/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.2838 - val_loss: 3.0477
Epoch 9/10
40875/40875 [==============================] - 55s 1ms/step - loss: 1.2678 - val_loss: 3.1436
Epoch 10/10
40875/40875 [==============================] - 56s 1ms

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

  63/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 4s 826us/step
              precision    recall  f1-score   support

           0       0.25      0.35      0.29      1499
           1       0.23      0.30      0.26      1499
           2       0.26      0.21      0.23      1499
           3       0.52      0.24      0.32      1499
           4       0.83      0.79      0.81      1499
           5       0.38      0.44      0.40      1499
           6       0.63      0.77      0.70      1499
           7       0.68      0.49      0.57      1499
           8       0.29      0.17      0.21      1499
           9       0.42      0.44      0.43      1499
          10       0.24      0.23      0.24      1499
          11       0.30      0.38      0.34      1499
          12       0.32      0.28      0.30      1499
          13       0.19      0.23      0.21      1499
          14       0.17      0.09      0.12      1499
          15       0.60      0.35      0.44      1499
          16       0.3

## 0-32

In [25]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [26]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_1457/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_1457/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_1457/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [27]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10


2023-10-04 15:33:30.818295: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 167424000 exceeds 10% of free system memory.


40875/40875 [==============================] - 57s 1ms/step - loss: 1.6556 - val_loss: 2.3405
Epoch 2/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.8721 - val_loss: 2.2248
Epoch 3/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.7078 - val_loss: 2.2364
Epoch 4/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.6260 - val_loss: 2.1327
Epoch 5/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.5750 - val_loss: 2.0719
Epoch 6/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.5402 - val_loss: 2.0871
Epoch 7/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.5111 - val_loss: 2.0639
Epoch 8/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.4882 - val_loss: 2.1843
Epoch 9/10
40875/40875 [==============================] - 56s 1ms/step - loss: 0.4700 - val_loss: 2.2029
Epoch 10/10
40875/40875 [==============================] - 56s 1ms

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

  62/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 4s 826us/step
              precision    recall  f1-score   support

           0       0.57      0.78      0.66      1499
           1       0.51      0.84      0.64      1499
           2       0.44      0.69      0.54      1499
           3       0.56      0.36      0.43      1499
           4       0.91      0.96      0.93      1499
           5       0.84      0.86      0.85      1499
           6       0.78      0.92      0.84      1499
           7       0.77      0.77      0.77      1499
           8       0.51      0.67      0.58      1499
           9       0.60      0.69      0.64      1499
          10       0.80      0.59      0.68      1499
          11       0.95      0.93      0.94      1499
          12       0.69      0.57      0.62      1499
          13       0.69      0.52      0.59      1499
          14       0.53      0.56      0.55      1499
          15       0.84      0.59      0.69      1499
          16       0.7

## 0-64

In [31]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [32]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10


2023-10-04 15:42:58.635284: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 334848000 exceeds 10% of free system memory.


40875/40875 [==============================] - 58s 1ms/step - loss: 1.2024 - val_loss: 2.0887
Epoch 2/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.5169 - val_loss: 1.7879
Epoch 3/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.3727 - val_loss: 1.8055
Epoch 4/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.3020 - val_loss: 1.6184
Epoch 5/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.2575 - val_loss: 1.5876
Epoch 6/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.2276 - val_loss: 1.5390
Epoch 7/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.2035 - val_loss: 1.6749
Epoch 8/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.1865 - val_loss: 1.4845
Epoch 9/10
40875/40875 [==============================] - 57s 1ms/step - loss: 0.1729 - val_loss: 1.4485
Epoch 10/10
40875/40875 [==============================] - 57s 1ms

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

 313/5106 [>.............................] - ETA: 3s

5106/5106 [==============================] - 4s 831us/step
              precision    recall  f1-score   support

           0       0.72      0.93      0.81      1499
           1       0.90      0.90      0.90      1499
           2       0.64      0.81      0.72      1499
           3       0.92      0.86      0.89      1499
           4       0.97      0.99      0.98      1499
           5       0.96      0.97      0.97      1499
           6       0.96      0.97      0.96      1499
           7       0.89      0.89      0.89      1499
           8       0.52      0.86      0.65      1499
           9       0.64      0.75      0.69      1499
          10       0.74      0.68      0.71      1499
          11       0.96      0.93      0.95      1499
          12       0.86      0.77      0.81      1499
          13       0.75      0.86      0.80      1499
          14       0.77      0.79      0.78      1499
          15       0.97      0.82      0.89      1499
          16       0.8